In [1]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import itertools
from IPython.display import display

## load network, clean duplicates, convert to nx format

In [2]:
df = pd.read_csv("train.csv")  # duplicate edges!
print df.shape
df.head()

(93200, 2)


,node1,node2
0,0,17
1,0,39
2,0,43
3,0,63
4,0,218


In [3]:
df = df[df.node1 < df.node2]
print df.shape
df.head()

(46600, 2)


,node1,node2
0,0,17
1,0,39
2,0,43
3,0,63
4,0,218


In [4]:
df.columns = ['source', 'target']
G = nx.from_pandas_edgelist(df).to_undirected()

## explore network

In [5]:
G.number_of_nodes()

1434

In [6]:
G.number_of_edges()

46600

# Features

## len shortest path

In [7]:
shortest_path_len = pd.DataFrame(nx.floyd_warshall_numpy(G), index=G.nodes(), columns=G.nodes())
shortest_path_len.head()

,0,1,2,3,4,5,6,7,8,9,...,1424,1425,1426,1427,1428,1429,1430,1431,1432,1433
0,0.0,2.0,2.0,3.0,3.0,3.0,3.0,2.0,3.0,3.0,...,3.0,3.0,4.0,3.0,2.0,2.0,3.0,2.0,2.0,3.0
1,2.0,0.0,2.0,3.0,1.0,3.0,1.0,2.0,1.0,3.0,...,2.0,2.0,3.0,2.0,3.0,2.0,3.0,2.0,2.0,2.0
2,2.0,2.0,0.0,2.0,2.0,3.0,2.0,1.0,2.0,3.0,...,3.0,3.0,3.0,3.0,1.0,2.0,2.0,2.0,2.0,3.0
3,3.0,3.0,2.0,0.0,3.0,3.0,3.0,2.0,3.0,3.0,...,4.0,3.0,4.0,3.0,2.0,2.0,3.0,2.0,3.0,4.0
4,3.0,1.0,2.0,3.0,0.0,3.0,2.0,2.0,2.0,3.0,...,2.0,2.0,3.0,2.0,2.0,2.0,2.0,3.0,1.0,2.0


In [8]:
features = pd.DataFrame(shortest_path_len.stack())
features.columns = ['shortest_path_len']
features = features[features.index.get_level_values(0) < features.index.get_level_values(1)]
print features.shape
features.head()

(1027461, 1)


shortest_path_len
0 1                2.0
  2                2.0
  3                3.0
  4                3.0
  5                3.0

## num of shared neighbors

In [9]:
features['num_common_neigh'] = features.index.map(lambda ind: len({x for x in G[ind[0]]}.intersection({x for x in G[ind[1]]})))

In [10]:
features.head()

shortest_path_len  num_common_neigh
0 1                2.0                 1
  2                2.0                 1
  3                3.0                 0
  4                3.0                 0
  5                3.0                 0

## has known path?

In [11]:
features['has_path'] = features.index.map(lambda ind: nx.has_path(G, ind[0], ind[1]))
features.head(3)

shortest_path_len  num_common_neigh has_path
0 1                2.0                 1     True
  2                2.0                 1     True
  3                3.0                 0     True

## resource_allocation_index

In [12]:
features.index.rename(['node1','node2'], inplace=True)

In [13]:
rai = pd.DataFrame([x for x in nx.resource_allocation_index(G, features.index)],
                  columns=['node1','node2','resource_allocation_index']).set_index(['node1','node2'])

In [14]:
features = features.join(rai)

## Jaccard coefficient (fraction of shared neighbors)

In [15]:
jaccard = pd.DataFrame([x for x in nx.jaccard_coefficient(G)], 
                       columns=['node1', 'node2', 'jaccard']).set_index(['node1','node2'])

In [16]:
features = features.join(jaccard)

## adamic_adar_index

In [17]:
adamic = pd.DataFrame([x for x in nx.adamic_adar_index(G)], 
                       columns=['node1', 'node2', 'adamic']).set_index(['node1','node2'])
features = features.join(adamic)

## preferential_attachment

In [18]:
preferential = pd.DataFrame([x for x in nx.preferential_attachment(G)], 
                       columns=['node1', 'node2', 'preferential']).set_index(['node1','node2'])
features = features.join(preferential)

## cn_soundarajan_hopcroft

In [19]:
# cn_soundarajan_hopcroft = pd.DataFrame([x for x in nx.preferential_attachment(G)], 
#                        columns=['node1', 'node2', 'cn_soundarajan_hopcroft']).set_index(['node1','node2'])
# features = features.join(cn_soundarajan_hopcroft)

## ra_index_soundarajan_hopcroft

In [20]:
# ra_index_soundarajan_hopcroft(G, ebunch=None, community='community')

## within_inter_cluster

In [21]:
# within_inter_cluster(G, ebunch=None, delta=0.001, community='community')

In [22]:
features.to_pickle('features.df')